In [ ]:
import os
import json
import time
import random
from tqdm import tqdm
import textwrap
import requests
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#project_id = 5795
project_id = 5654

# Projet general data

In [ ]:
url = 'https://tasks.hotosm.org/api/v1/project/' + str(project_id)
headers = {'Accept': 'application/json', 'Authorization': 'Token sessionTokenHere==', 'Accept-Language': 'en'}
r = requests.get(url, headers=headers)
%time data_project = r.json()

### List task ids and store them

In [ ]:
tasks_ids = list()
for feature in data_project['tasks']['features']:
    tasks_ids.append(feature['properties']['taskId'])
data_project['tasks_ids'] = tasks_ids

In [ ]:
tasks_ids[:10]

### Add some summary data

In [ ]:
url = 'https://tasks.hotosm.org/api/v1/project/' + str(project_id) + '/summary'    
r = requests.get(url, headers=headers)
summary = r.json()

In [ ]:
summary

In [ ]:
for key in summary:
    if key not in data_project.keys():
        data_project[key] = summary[key]

### Add task history data

In [ ]:
task_history = dict()
missing_taks = list()
for task_id in tqdm(tasks_ids):
    url = 'https://tasks.hotosm.org/api/v1/project/' + str(project_id) + '/task/' + str(task_id)    
    r = requests.get(url, headers=headers)
    if r.ok :
        task_history[task_id] = r.json()
    else :
        missing_taks.append(task_id)
    time.sleep(0.5 + random.random())
print(f'{len(missing_taks)} missing tasks')

In [ ]:
data_project['task_history'] = task_history

### Store data

In [ ]:
with open(str(project_id) + '.json', 'w') as outfile: 
    json.dump(data_project, outfile)